In [2]:
## pip install moviepy==1.0.0

import os
import librosa
import pandas as pd
from moviepy.editor import *
from tqdm import tqdm_notebook
import numpy as np

In [5]:
#from utils import make_dir
def make_dir(dir_):
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

In [6]:
audio_label_clip_dir = os.path.join('.', 'data', 'audio_label_clip')
make_dir(audio_label_clip_dir)

In [7]:
def save(audios, label):
    data = pd.DataFrame({
        'audios':audios, 'label':label })
    data.to_csv(audio_label_clip_dir+'/audio_label_clip.csv', index=False, encoding='euc-kr')

In [9]:
## 레이블링 데이터 위치 : ./data/label
files = []
for file in os.listdir(os.path.join('.', 'data', 'label')):
    _, file_id = os.path.split(file)
    if file_id.endswith('srt'):
        name = file_id.split(".")[0]
        if(name not in files):
            files.append(name)

In [10]:
audio = []
label = []
shi = 0
jon = 0
sha = 0
byoung = 0
background = 0
negative = 0
count = 0

for file in files:
    print("file name:" + file)
    text = './data/label/' + file + ".txt"
    aud = './data/label/' + file + ".wav"
    print("read new file")
    data = pd.read_csv(text, sep="\t", names=['start', 'end', 'label'])
    print(data)
#     data.start.astype('float')
#     data.end.astype('float')
    for i in range(len(data)):
        lab = data.label[i]
        print(data.start[i], data.end[i], data.label[i])
        try:
            start = float(data.start[i])
            end = float(data.end[i])
        except ValueError:
            continue
        if start >= end:
            continue
        if np.array(data.label[i])==np.nan:
            continue
        full_audio = AudioFileClip(aud)
        if(lab==10): #shi
            name = "shi_" + str(shi)
            shi += 1
        elif(lab==11): #jon 
            name = "jon_" + str(jon)
            jon += 1
        elif(lab==12): #sha
            name = "sha_" + str(sha)
            sha += 1
        elif(lab==13): #byoung
            name = "byoung_" + str(byoung)
            byoung += 1
        elif(lab==9): #background
            name = "background_" + str(background)
            background+= 1
        elif(lab==0): #negative
            name = "negative_" + str(negative)
            negative += 1
        tmp_audio = full_audio.subclip(start, end)
        name = os.path.join(audio_label_clip_dir,  name + ".wav")
        try:
            tmp_audio.write_audiofile(name)
        except OSError:
            print('oserror')
            continue
            
        audio.append(name + ".wav")
        label.append(int(lab))
        print('dd')
    count += 1
save(audio, label)